In [0]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Выведем список категорий текстов:

In [23]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Атрибут ```traget``` хранит номера категорий для текстов из обучающей выборки:

In [24]:
newsgroups_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

Доступ к самим текстам через атрибут ```data```. Выведем текст и категорию случайного примера из обучающего датасета:

In [25]:
n = 1006
print('Topic = {0}\n'.format(newsgroups_train.target_names[newsgroups_train.target[n]]))
print(newsgroups_train.data[n])

Topic = talk.religion.misc

Tony-

I read your post, it was nothing new, I had seen much the same in other
typical"Christian" anti-gay sentimental literature.  Gay people are and will
con- tinue to be persecuted as long as such propaganda petpetuates.  You may
be unaware of all the statistica "findings" concerning African-Americans that
have been published and used by various groups to re-enforce their own bias
against African-Americans.  We usually think of the KKK in these instances,
but there are many other groups.  Of course, the vast majority of the public
scoff at such findings and documents today, but that was not always the case.
Fortunately African-Americans had "whites" who supported their 'cause' and
public sentiment was eventually (if not entirely) turned around.  There was
even a Civil War, and anti-negro sentiment increased.  In fact, until laws
were put in place to protect the inalienable rights of Blacks it was pretty
much legal to discriminate against them.

I know man

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True, min_df=8, max_df=0.04)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.04, max_features=None, min_df=8,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}),
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [27]:
vectorizer.vocabulary_

{'wondering': 12090,
 'enlighten': 4311,
 'car': 2365,
 'saw': 9812,
 'door': 3971,
 'sports': 10472,
 'looked': 6855,
 'late': 6603,
 '60s': 690,
 'early': 4106,
 '70s': 762,
 'doors': 3972,
 'small': 10300,
 'addition': 1076,
 'bumper': 2229,
 'separate': 9995,
 'rest': 9464,
 'body': 2035,
 'model': 7367,
 'engine': 4295,
 'specs': 10430,
 'production': 8752,
 'history': 5590,
 'info': 5989,
 'fair': 4640,
 'brave': 2127,
 'souls': 10382,
 'upgraded': 11600,
 'si': 10160,
 'clock': 2698,
 'oscillator': 8008,
 'shared': 10072,
 'experiences': 4530,
 'poll': 8508,
 'brief': 2154,
 'message': 7216,
 'detailing': 3694,
 'procedure': 8730,
 'speed': 10436,
 'cpu': 3268,
 'rated': 9094,
 'add': 1072,
 'cards': 2373,
 'adapters': 1069,
 'heat': 5511,
 'sinks': 10221,
 'hour': 5683,
 'usage': 11624,
 'floppy': 4862,
 'disk': 3863,
 'functionality': 5042,
 '800': 801,
 'floppies': 4861,
 'especially': 4393,
 'requested': 9404,
 'summarizing': 10766,
 'days': 3476,
 'network': 7652,
 'knowled

Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы.


In [0]:
from tqdm import tqdm
def lda(n, X_train):
    nk=np.zeros(n)
    ndk=np.zeros((X_train.shape[0], n))
    nkw=np.zeros((n, X_train.shape[1]))

    doc, words = X_train.nonzero()
    z=np.random.choice(n, len(doc))

    for d, word, k in zip(doc, words, z):
        ndk[d,k]+=1
        nkw[k,word]+=1
        nk[k]+=1

    beta=np.ones(X_train.shape[1])
    alpha=np.ones(n)
    
    for j in tqdm(range(50)):
        for i in range(len(doc)):
            d_i=doc[i]
            z_i=z[i]
            w_i=words[i]
            nkw[z_i, w_i]-=1
            ndk[d_i,z_i]-=1
            nk[z_i]-=1
            
            p = (ndk[d_i, :]+alpha)*(nkw[:,w_i]+beta[w_i])/(nk+beta.sum())
            z[i] = np.random.choice(n, p = p/p.sum())
            
            nkw[z[i],w_i]+=1
            ndk[d_i, z[i]]+=1
            nk[z[i]]+=1
            
            
    return nkw

In [33]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.nnz / np.prod(X_train.shape)
nkw=lda(20, X_train)

100%|██████████| 50/50 [30:03<00:00, 35.62s/it]


In [34]:
#Let's take a look at the most likely words from topics 1, ..., 20
n = np.argsort(nkw)[:, -10:]
n = n[:, ::-1]
t = np.zeros((20,X_train.shape[1]))
for k in range(20):
    for i in n[k]:
        t[k][i]=1
    print('Topic {}:\t{}'.format(k+1, '\t'.join(vectorizer.inverse_transform(t)[k])))

Topic 1:	game	games	hockey	league	play	players	season	team	teams	win
Topic 2:	1993	center	earth	general	nasa	research	science	space	technology	university
Topic 3:	banks	chastity	geb	gordon	intellect	pitt	shameful	skepticism	soon	surrender
Topic 4:	14	25	34	hi	mb	mi	mr	ms	ok	tm
Topic 5:	chip	clipper	encryption	key	keys	phone	public	secret	secure	security
Topic 6:	application	code	file	files	ftp	running	server	version	window	write
Topic 7:	11	12	13	14	16	17	18	19	24	25
Topic 8:	bike	car	cars	engine	miles	nice	ride	road	speed	turn
Topic 9:	certainly	hear	love	mentioned	sea	sound	stay	stuff	tek	vice
Topic 10:	100	asking	buy	condition	offer	original	price	sale	sell	shipping
Topic 11:	children	history	israel	israeli	jewish	jews	killed	land	today	war
Topic 12:	air	circuit	current	electronics	goes	hot	low	stuff	voltage	water
Topic 13:	american	clinton	control	country	crime	gun	law	public	rights	states
Topic 14:	address	advance	anybody	appreciate	deleted	feel	hear	interesting	posting	sorry
Topi

По большей части наиболее вероятных слов восстанавливается их тема. 
```
Topic 1:rec.sport.hockey
Topic 2:sci.space
Topic 4:comp.os.ms-windows.misc
Topic 5:sci.crypt
Topic 6:comp.windows.x
Topic 8:rec.autos
Topic 10:misc.forsale
Topic 11:talk.politics.mideast
Topic 12:sci.electronics
Topic 13:talk.politics.guns
Topic 16:talk.politics.misc
Topic 18:soc.religion.christian
Topic 19:comp.sys.mac.hardware
Topic 20:sci.med
```

In [35]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']